In [ ]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter('ignore')

In [ ]:
# !wget https://raw.githubusercontent.com/huseinzol05/Malaya-Dataset/master/200k-english-malay/200k-english-malay.json

In [ ]:
import json
with open('200k-english-malay.json') as fopen:
    english_malay = json.load(fopen)

malay = [i[1] for i in english_malay if i[0] != i[1] and len(i[1]) < 15]
len(malay)

In [ ]:
malay[:10]

In [ ]:
import requests
import imgkit
from scipy.misc import imread, imsave
from tqdm import tqdm
import threading
from queue import Queue

In [ ]:
def save_image(word):
    payload = {'phonText':word}
    r = requests.post('http://www.ejawi.net/result.php',data=payload)
    r = r._content.decode().replace('font face="ArialJawi" size="5"', 
                                    'font face="ArialJawi" size="5" style="font-size:100;padding-right:50px"')
    with open('cache/%s.html'%(word),'w') as f:
        f.write(r)
    imgkit.from_file('cache/%s.html'%(word), 'cache/%s.jpg'%(word), options = {"xvfb": "","quiet": ""})
    img = imread('cache/%s.jpg'%(word))
    img = img[:, img.shape[1] // 4:]
    imsave('1.png',img)
    imsave('words/%s.png'%(word), img)
    return 'Done'
    
def run_parallel_in_threads(args_list, target = save_image):
    globalparas = []
    result = Queue()

    def task_wrapper(*args):
        result.put(target(*args))

    threads = [
        threading.Thread(target = task_wrapper, args = args)
        for args in args_list
    ]
    for t in threads:
        t.start()
    for t in threads:
        t.join()
    while not result.empty():
        globalparas.append(result.get())

In [ ]:
batch_size = 10

for no in tqdm(range(0, len(malay), batch_size)):
    data = malay[no : no + batch_size]
    combined = [
        (data[i],) for i in range(len(data))
    ]
    run_parallel_in_threads(combined)

In [ ]:
!tar -czvf jawi-rumi.tar.gz words